In [1]:
from io import BytesIO
from base64 import b64encode
import rasterio as rio
import matplotlib
import matplotlib.cm as cm
import ipywidgets as widgets
from ipyleaflet import Map, Marker, ImageOverlay
import rasterio as rio
from PIL import Image

center = (53.8008, -1.5491)
m = Map(center=center, zoom=15)

ndvi_array = rio.open('Data/leeds_NDVI_aug_highres.tif').read()
ndwi_array = rio.open('Data/leeds_NDWI_aug_highres.tif').read()
ndbi_array = rio.open('Data/leeds_NDBI_aug_highres.tif').read()
raster_reader = rio.open('Data/leeds_NDBI_aug_highres.tif')
raster_transform = raster_reader.transform
bounds = raster_reader.bounds
SW = (bounds.bottom, bounds.left)
NE = (bounds.top, bounds.right)
bounds_tuple = (SW, NE)




def model_airtemp(solar_irradiance, ndvi, ndbi, ndwi, c=25, ndvi_beta=-3, ndbi_beta=4, ndwi_beta=-2):
    airtemp =  ndvi_beta*ndvi + ndbi_beta*ndbi + ndwi_beta*ndwi + np.random.normal(-1, 1) + c + solar_irradiance
    return airtemp


def encode_array(array, filename='temp.png'):
    array = np.moveaxis(array, 0, -1)
    nan_mask = ~np.isnan(array) * 1 
    nan_mask *= 255
    nan_mask = nan_mask.astype(np.uint8)
    array_max = np.nanmax(array)
    array_min = np.nanmin(array)


    array = np.nan_to_num(array)


    array = np.clip((array - array_min) / (array_max - array_min) * 255, 0, 255)
    array = array.astype(np.uint8)
    
    image = Image.fromarray(np.squeeze(np.stack([array, array, array, nan_mask], axis=-1)), mode="RGBA")

    
    #Convert the image to bytes and encode in the url
    s = BytesIO()
    image.save(s, 'png')
    data = b64encode(s.getvalue())
    data = data.decode('ascii')
    imgurl = 'data:image/png;base64,' + data
    
    return imgurl


def image_to_8bit(array):
    array_max = np.nanmax(array)
    array_min = np.nanmin(array)

    array = np.nan_to_num(array)


    array = np.clip((array - array_min) / (array_max - array_min) * 255, 0, 255)
    array = array.astype(np.uint8)
    return array 


In [8]:
import geopandas as gpd
vector = gpd.read_file("Data/leeds_census_final.geojson", driver='GeoJSON')




In [26]:
import random
random_data_button = widgets.Button(description= 'Randomise!')
def randomise(b):
    vector['Random'] = vector['Population'] * random.random()
    display(display_chart(vector))

frameDis = widgets.Output()

random_data_button.on_click(randomise)


In [10]:
from rasterstats import zonal_stats
import numpy as np

def hazard_score(vector, raster, transform):
    stats = zonal_stats(vector, np.squeeze(raster), affine=transform)
    mean_list = []
    
    for s in stats:
        mean_list.append(s['mean'])
        
    vector['Mean Temp'] = mean_list
    vector['Hazard Score'] = vector['Mean Temp']*vector['Population']
    # Now we'll normalise the hazard score 
    vector['Hazard Score'] /= vector['Hazard Score'].max()
    return vector


In [25]:
solar_irradiance_slider = widgets.FloatSlider(value=14, min=10, max=20, step=0.1, description='Solar Irradiance')
ndvi_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='NDVI Coefficient')
ndbi_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='NDBI Coefficient')
ndwi_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='NDWI Coefficient')

histDis = widgets.Output()

def display_chart(vector):
    with frameDis:
        display(vector[['LSOA Code', 'Hazard Score']], clear=True)  
    with histDis:
        histDis.clear_output()
        _h = plt.hist(vector['Hazard Score'])
        plt.show(_h)

#data_hbox = widgets.HBox([histDis, frameDis])

# As before, we will define a callback function which attach
# to the observer of the leaflet map, but now we add
# the frame display widget as well 

frameDis = widgets.Output()

def update_map(change):

    new_airtemp = model_airtemp(solar_irradiance_slider.value, ndvi_array, ndbi_array, ndwi_array, 13, ndvi_slider.value, 
                  ndbi_slider.value, ndwi_slider.value)
    
    path = encode_array(new_airtemp)

    ## Now we add the code to update the dataframe
    # If we declare a variable in a function, we cant
    # access a global variable with the same name
    # so we are assigning the new vector to _vector
    _vector = hazard_score(vector, new_airtemp, raster_transform)
    display_chart(_vector)

    imageLayer = ImageOverlay(url=path, bounds=bounds_tuple)
    
    #Remove old layers
    try:
        for layer in m.layers[1:]:
            m.remove_layer(layer)
    except:
        pass    

    
    m.add_layer(imageLayer)



solar_irradiance_slider.observe(update_map, 'value')
ndvi_slider.observe(update_map, 'value')
ndbi_slider.observe(update_map, 'value')
ndwi_slider.observe(update_map, 'value')

irradiance_slider_container = widgets.Box([solar_irradiance_slider])
parameters_container = widgets.VBox([ndvi_slider, ndbi_slider, ndwi_slider])

controls_container = widgets.VBox([irradiance_slider_container, parameters_container])
mapDisplay = widgets.Output()

mapLayout = widgets.HBox([m, controls_container])
#We will add the frame display beneath the map
data_hbox = widgets.HBox([frameDis, histDis])

mapChartLayout = widgets.VBox([mapLayout, data_hbox])
display(mapChartLayout)

